# 2.1.1 Zeichenerklärung – OCR (GPT-4o)

* Zero-Shot, Chain-Of-Thought
* Beste Ergebnisse (img_prompts): A3, B1, B2

In [2]:
from utils.openai import OpenAI
from utils.parser import Parser
from utils.runner import Runner

instructions = "Du bist ein Assistent zur getreuen Wiedergabe von Informationen aus einem Bebauungsplan. Achte auf Vollständigkeit."
jarvis = OpenAI(instructions)
parser = Parser()
runner = Runner()

### Z1) Schema Nutzungsschablone

In [2]:
# Z1.1) Zeichenerklärung – Schema Nutzungsschablone WITHOUT BOUNDING BOX
ze_tt_path = "../data/processed/bpläne/1_alles_in_einem_dokument/L04-ZE-TT-crop.png"
ze_path = "../data/processed/bpläne/1_alles_in_einem_dokument/L04-ZE-crop.png"
ze_two_col_path = "../data/processed/bpläne/1_alles_in_einem_dokument/S03-ZE-two-col-crop.png"

instruction = 'Gehe Schritt für Schritt vor: 1. Finde den Eintrag "Nutzungsschablone" in der vorliegenden Zeichenerklärung. 2. Finde die zugehörige Tabellenerklärung zum Eintrag "Nutzungsschablone". 3. Gebe die Tabellenerklärung in ASCII-Tabellenform aus.'

async def run():
    return await jarvis.extractTextFromImages(instruction, [ze_tt_path, ze_path, ze_two_col_path])
await runner.async_consistency_check(run)

['1. **Eintrag "Nutzungsschablone" in der Zeichenerklärung:**

In der Zeichenerklärung auf der linken Seite des Bebauungsplans gibt es einen Eintrag "Nutzungsschablone". Dieser Eintrag ist durch ein Symbol dargestellt, das wie folgt aussieht:

```
[Symbol]
Nutzungsschablone
```

2. **Zuordnung der Tabellenerklärung:**

Die zugehörige Tabellenerklärung zum Eintrag "Nutzungsschablone" befindet sich auf der rechten Seite des Bebauungsplans unter dem Abschnitt "Planungsrechtliche Festsetzungen". Der relevante Abschnitt ist "2.0 Nutzungsschablone".

3. **Tabellenerklärung in ASCII-Tabellenform:**

```
+----------------------+---------------------------------------------+
| Nutzungsschablone    | Festsetzungen                               |
+----------------------+---------------------------------------------+
| 2.0                  | Nutzungsschablone ist die im Plan           |
|                      | eingetragene Nutzung.                       |
+----------------------+-----------------

In [2]:
# Z1.2) Zeichenerklärung – Schema Nutzungsschablone WITH BOUNDING BOX
ze_tt_bb_path = "../data/processed/bpläne/1_alles_in_einem_dokument/L04-ZE-TT-bb-crop.png"
ze_bb_path = "../data/processed/bpläne/1_alles_in_einem_dokument/L04-ZE-bb-crop.png"
ze_two_col_bb_path = "../data/processed/bpläne/1_alles_in_einem_dokument/S03-ZE-two-col-bb-crop.png"

instruction = 'Gebe die rot umrandete Tabellenerklärung zum Eintrag Nutzungsschablone in ASCII-Tabellenform aus.'

async def run():
    return await jarvis.extractTextFromImages(instruction, [ze_tt_bb_path, ze_bb_path, ze_two_col_bb_path])
await runner.async_consistency_check(run)

['Hier ist die rot umrandete Tabellenerklärung zum Eintrag Nutzungsschablone in ASCII-Tabellenform:

```
+------------------+----------------------+
| Baugebiet        | Zahl der Vollgeschosse|
+------------------+----------------------+
| Grundflächenzahl | Geschossflächenzahl  |
+------------------+----------------------+
| Bauweise         |                      |
+------------------+----------------------+
| Dachform und     |                      |
| Dachneigung      |                      |
+------------------+----------------------+
```', 'Hier ist die rot umrandete Tabellenerklärung zum Eintrag Nutzungsschablone in ASCII-Tabellenform:

```
+----------------+---------------------+
| Baugebiet      | Zahl der Vollgeschosse |
+----------------+---------------------+
| Grundflächenzahl | Geschossflächenzahl  |
+----------------+---------------------+
| Bauweise       |                     |
+----------------+---------------------+
| Dachform und Dachneigung             |
+---------

In [13]:
# Z1.3) Zeichenerklärung – Schema Nutzungsschablone CROPPED
ze_l04_nz_path = "../data/processed/bpläne/1_alles_in_einem_dokument/L04-ZE-nz-crop.png"
ze_s03_nz_path = "../data/processed/bpläne/1_alles_in_einem_dokument/S03-ZE-nz-crop.png"

instruction = 'Gebe das Schema der Tabellenerklärung (Nutzungsschablone) strukturiert in ASCII-Tabellenform aus.'

async def run():
    return await jarvis.extractTextFromImages(instruction, [ze_l04_nz_path, ze_s03_nz_path])
results = await runner.async_consistency_check(run)
msg211_l04_nz = results[0][0]
msg211_s03_nz = results[0][1]
%store msg211_l04_nz msg211_s03_nz

['Hier ist das Schema der Tabellenerklärung in ASCII-Tabellenform:

```
+---------------------+--------------------------+
| Baugebiet           | Zahl der Vollgeschosse   |
+---------------------+--------------------------+
| Grundflächenzahl    | Geschossflächenzahl      |
+---------------------+--------------------------+
| -                   | Bauweise                 |
+---------------------+--------------------------+
| Dachform und Dachneigung                       |
+-----------------------------------------------+
```', 'Hier ist das Schema der Tabellenerklärung in ASCII-Tabellenform:

```
+-----------------------------+-----------------------------+
| Baugebiet                   | Zahl d. Vollgeschosse       |
+-----------------------------+-----------------------------+
| Grundflächenzahl            | Geschossflächenzahl         |
+-----------------------------+-----------------------------+
| Baumassenzahl               | Bauweise                    |
+--------------------

### Z2) Zeichenerklärung der Nutzungsschablone

In [3]:
# Z2.1) Zeichenerklärung – Inhalt zu den Hauptkategorien – NO CONTEXT
ze_path = "../data/processed/bpläne/1_alles_in_einem_dokument/L04-ZE-crop.png"
ze_two_col_path = "../data/processed/bpläne/1_alles_in_einem_dokument/S03-ZE-two-col-crop.png"
images = [ze_path, ze_two_col_path]

instruction = 'Extrahiere alle Informationen zu den folgenden Themen: Art der baulichen Nutzung, Maß der baulichen Nutzung, Bauweise, überbaubare Grundstücksfläche.'

async def run():
    return await jarvis.extractTextFromImages(instruction, images)
await runner.async_consistency_check(run)

['Hier sind die extrahierten Informationen zu den angeforderten Themen aus dem Bebauungsplan:

**Art der baulichen Nutzung (§ 9 (1) 1 BBauG):**
- **Wohnbauflächen:**
  - **WR:** Reines Wohngebiet (§ 3 BauNVO)
  - **WA:** Allgemeines Wohngebiet (§ 4 BauNVO)
- **Mischgebiet:**
  - **MI:** Mischgebiet (§ 6 BauNVO)
- **Gewerbegebiet:**
  - **GE2:** Eingeschränktes Gewerbegebiet für nicht wesentlich störende Gewerbebetriebe (§ 8 BauNVO)

**Maß der baulichen Nutzung (§ 9 (1) 1 b BBauG):**
- **Zahl der Vollgeschosse (§ 18 BauNVO)**
- **Grundflächenzahl (GRZ) (§ 19 BauNVO)**
- **Geschossflächenzahl (GFZ) (§ 20 BauNVO)**

**Bauweise (§ 1 (1) 1b BBauG und § 22 BauNVO):**
- **Offene Bauweise:** Nur Einzel- und Doppelhäuser zulässig
- **Geschlossene Bauweise**

**Überbaubare Grundstücksflächen (§ 9 (1) 1b BBauG):**
- **Baugrenze (§ 23 (3) BauNVO)**

Diese Informationen geben einen Überblick über die Festsetzungen im Bebauungsplan bezüglich der Art und des Maßes der baulichen Nutzung, der Bauweise 

[['Hier sind die extrahierten Informationen zu den angeforderten Themen aus dem Bebauungsplan:\n\n**Art der baulichen Nutzung (§ 9 (1) 1 BBauG):**\n- **Wohnbauflächen:**\n  - **WR:** Reines Wohngebiet (§ 3 BauNVO)\n  - **WA:** Allgemeines Wohngebiet (§ 4 BauNVO)\n- **Mischgebiet:**\n  - **MI:** Mischgebiet (§ 6 BauNVO)\n- **Gewerbegebiet:**\n  - **GE2:** Eingeschränktes Gewerbegebiet für nicht wesentlich störende Gewerbebetriebe (§ 8 BauNVO)\n\n**Maß der baulichen Nutzung (§ 9 (1) 1 b BBauG):**\n- **Zahl der Vollgeschosse (§ 18 BauNVO)**\n- **Grundflächenzahl (GRZ) (§ 19 BauNVO)**\n- **Geschossflächenzahl (GFZ) (§ 20 BauNVO)**\n\n**Bauweise (§ 1 (1) 1b BBauG und § 22 BauNVO):**\n- **Offene Bauweise:** Nur Einzel- und Doppelhäuser zulässig\n- **Geschlossene Bauweise**\n\n**Überbaubare Grundstücksflächen (§ 9 (1) 1b BBauG):**\n- **Baugrenze (§ 23 (3) BauNVO)**\n\nDiese Informationen geben einen Überblick über die Festsetzungen im Bebauungsplan bezüglich der Art und des Maßes der bauliche

In [7]:
# Z2.2) Zeichenerklärung – Inhalt zu den Hauptkategorien – WITH CONTEXT
# Kontext: Schema, BauNVO
%store -r msg211_l04_nz msg211_s03_nz msgbaunvo
images = [ze_path, ze_two_col_path]
schemas = parser.text2prompts([msg211_l04_nz, msg211_s03_nz])
baunvo = parser.text2prompts([msgbaunvo])
contexts = list(map(lambda schema: [*baunvo,schema], schemas))

instruction = 'Extrahiere alle Informationen aus der Zeicheerklärung zu den folgenden Themen: Art der baulichen Nutzung, Maß der baulichen Nutzung, Bauweise, überbaubare Grundstücksfläche.'

async def run():
    return await jarvis.extractTextFromImagesWithContexts(instruction, images, contexts)
results = await runner.async_consistency_check(run)
msg211_l04_ze = results[0][0]
%store msg211_l04_ze

['### Art der baulichen Nutzung

- **Wohnbauflächen**
  - **WR**: Reines Wohngebiet (§ 3 BauNVO)
  - **WA**: Allgemeines Wohngebiet (§ 4 BauNVO)
- **Gemischte Bauflächen**
  - **MI**: Mischgebiet (§ 6 BauNVO)
  - **GE2**: Eingeschränktes Gewerbegebiet für nicht wesentlich störende Gewerbebetriebe (§ 8 BauNVO)

### Maß der baulichen Nutzung

- **Zahl der Vollgeschosse** (§ 18 BauNVO)
- **Grundflächenzahl (GRZ)** (§ 19 BauNVO)
- **Geschossflächenzahl (GFZ)** (§ 20 BauNVO)

### Bauweise

- **Offene Bauweise**: Nur Einzel- und Doppelhäuser zulässig
- **Geschlossene Bauweise**

### Überbaubare Grundstücksfläche

- **Baugrenze** (§ 23 (3) BauNVO)', '### Art der baulichen Nutzung

- **Kleinsiedlungsgebiet (WS)**
- **Reines Wohngebiet (WR)**
- **Allgemeines Wohngebiet (WA)**
- **Dorfgebiet (MD)**
- **Mischgebiet (MI)**
- **Kerngebiet (MK)**
- **Gewerbegebiet (GE)**
- **Industriegebiet (GI)**
- **Sondergebiet (SO)**
- **Flächen für den Gemeinbedarf (G)**:
  - Parkanlage
  - Sportplatz
  - Spiel